In [0]:

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pandas as pd

def get_date_list(start_date, end_date, freq="D"):
    import pandas as pd
    """
    freq:   D: calendar day frequency
            M: month end frequency
            MS: month start frequency
            A, Y: year end frequency
            AS, YS: year start frequency
    """
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list
    
dates = get_date_list('2010-07-04', '2010-07-31', freq='D')    # >> ['2010-07-04', '2010-07-05', '2010-07-06']
dates = dates + get_date_list('2010-08-01', '2020-04-18', freq='D')
# dates = dates + get_date_list('2010-08-01', '2010-09-30', freq='D')
print dates
# mapping:  ios-phone,    ios-all,     ios-tablet,  free_app_download=2, facebook
app_list = ['377194688', '364709193', '379174209',  '339739007',         '284882215', '20600000025034', '20600000357382', '20600000246936', '20600000009072']
device_code = ['ios-phone','ios-tablet','ios-all', 'android-all']    # exclude 'android-all'
countries = ['WW', 'US']
# app = [
#     ('377194688', 'ios-phone', 'WW'),
#     ('364709193', 'ios-all', 'US'),
#     ('379174209', 'ios-tablet', 'US'),
#     ('339739007', 'ios-phone', 'WW'),
#     ('284882215', 'ios-all', 'WW'),    # facebook
#     ('20600000025034', 'android-all', 'WW'),
#     ('20600000357382', 'android-all', 'WW'),
#     ('20600000246936', 'android-all', 'WW'),
#     ('20600000348221', 'android-all', 'TH'),
#     ('20600000009072', 'android-all', 'WW')
#     ]

def main_test(dates, app_list, device_code, countries):
    where_clause = "app_id in ({}) and device_code in ('{}') and country_code in ('{}')".format(",".join(map(str,app_list)),   "','".join(device_code), "','".join(countries) )
    # where_clause = "app_id={} and device_code={} and country_code={}".format(app_id, device_code, country_code)
    # >> "app_id in (377194688,364709193,379174209,339739007,284882215) and device_code in ('ios-phone','ios-tablet','ios-all') and country_code in ('WW','US')"
    print where_clause

    concat_list = []
    for date in dates:
    # for app_id, device_code, country_code in app:
        # where_clause = "app_id='{}' and device_code='{}' and country_code='{}'".format(app_id, device_code, country_code)
        unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/")\
        .parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))\
        .where(where_clause).select('app_id', 'device_code', 'country_code', 'free_app_download', 'paid_app_download', 'revenue').toPandas()
        
        concat_list.append(unified_data)
        
    temp_df = pd.concat(concat_list).groupby(['app_id', 'device_code', 'country_code'], sort=True).sum(level=['free_app_download', 'paid_app_download', 'revenue'])
    
    # fetch data from cumulative
    # cum_concat_list = []
    # for app_id, device_code, country_code in app:
    # where_clause = "app_id='{}' and device_code='{}' and country_code='{}'".format(app_id, device_code, country_code)
    cum_df = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}/".format(dates[-1])).where(where_clause).select('app_id', 'device_code', 'country_code', 'free_app_download', 'paid_app_download', 'revenue').toPandas()
    # cum_concat_list.append(cum_df)
    # cum_temp_df = pd.concat(cum_concat_list).groupby(['app_id', 'device_code', 'country_code'], sort=True).sum(level=['free_app_download', 'paid_app_download', 'revenue'])
    cum_temp_df = cum_df.groupby(['app_id', 'device_code', 'country_code'], sort=True).sum(level=['free_app_download', 'paid_app_download', 'revenue'])
    return temp_df, cum_temp_df

temp_df, cum_temp_df = main_test(dates, app_list, device_code, countries)
# print temp_df
# print cum_temp_df

# compare
import numpy as np
def temp_log_to_s3(log, name, file_format):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'zidong/aa.store.cum/{}.{}'.format(name, file_format))
    if file_format == 'json':
        s3object.put(Body=json.dumps(log))
        return json.dumps(log)
    else:
        s3object.put(Body=str(log))    
        return log

# print sorted(list(temp_df['free_app_download'])) == sorted(list(cum_df['free_app_download']))

# joined_df = temp_df.merge(cum_df, on=['app_id', 'device_code', 'country_code'])
# print joined_df.T

def _compare_df(df1, df2, on=None):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
        print diff_df
        diff_df = diff_df.loc[diff_df["_merge"] != "both"]
        print diff_df.empty
        if not diff_df.empty:
            print diff_type
            print diff_df
    temp_log_to_s3(diff_df, 'diff_df', 'csv')

# temp_df['free_app_download'] = [np.nan if i==5 else i for i in temp_df['free_app_download']]
# temp_df['free_app_download'] = [i if str(i).isdigit() else 0 for i in temp_df['free_app_download']]
# print [i.__repr__() for i in temp_df['free_app_download']]

temp_df[temp_df['free_app_download'].isnull()] = 0
# temp_df[temp_df['paid_app_download'].isnull()] = 0
# temp_df[temp_df['revenue'].isnull()] = 0

# cum_temp_df['free_app_download'] = [0 if l is None else l for l in cum_temp_df['free_app_download']]
# cum_temp_df['paid_app_download'] = [0 if l is None else l for l in cum_temp_df['paid_app_download']]
# cum_temp_df['revenue'] = [0 if l is None else l for l in cum_temp_df['revenue']]
# print temp_df

_compare_df(temp_df, cum_temp_df, on=['app_id', 'device_code', 'country_code', 'free_app_download'])
# diff_df = cum_temp_df.merge(temp_df, indicator=True, how='left', on=['app_id'])
# print diff_df.T

test_df = temp_df.reset_index()
# temp_df.drop(['index'], axis=1, inplace=True)
print test_df
# # save
spark.createDataFrame(test_df).write.parquet("s3://b2c-prod-data-pipeline-qa/zidong/aa.store.cum/store_cum_sample_result/", mode="overwrite")
spark.read.parquet("s3://b2c-prod-data-pipeline-qa/zidong/aa.store.cum/store_cum_sample_result/").show(100)

# # # compare
# import numpy as np
# def temp_log_to_s3(log, name, file_format):
#     import boto3
#     import json
#     s3 = boto3.resource('s3')
#     s3object = s3.Object('b2c-prod-data-pipeline-qa', 'zidong/aa.store.cum/{}.{}'.format(name, file_format))
#     if file_format == 'json':
#         s3object.put(Body=json.dumps(log))
#         return json.dumps(log)
#     else:
#         s3object.put(Body=str(log))    
#         return log

# # print sorted(list(temp_df['free_app_download'])) == sorted(list(cum_df['free_app_download']))

# # joined_df = temp_df.merge(cum_df, on=['app_id', 'device_code', 'country_code'])
# # print joined_df.T

# def _compare_df(df1, df2, on=None):
#     for diff_type in ["left", "right"]:
#         diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
#         diff_df = diff_df.loc[diff_df["_merge"] != "both"]
#         print diff_df.empty
#         if not diff_df.empty:
#             print diff_type
#             print diff_df
#     print "diff_df", diff_df
    

# # temp_df['free_app_download'] = [np.nan if i==5 else i for i in temp_df['free_app_download']]
# # temp_df['free_app_download'] = [i if str(i).isdigit() else 0 for i in temp_df['free_app_download']]
# # print [i.__repr__() for i in temp_df['free_app_download']]

# temp_df[temp_df['free_app_download'].isnull()] = 0
# temp_df[temp_df['paid_app_download'].isnull()] = 0
# temp_df[temp_df['revenue'].isnull()] = 0

# cum_temp_df['free_app_download'] = [0 if l is None else l for l in cum_temp_df['free_app_download']]
# cum_temp_df['paid_app_download'] = [0 if l is None else l for l in cum_temp_df['paid_app_download']]
# cum_temp_df['revenue'] = [0 if l is None else l for l in cum_temp_df['revenue']]
# # cum_temp_df.set_index(['app_id', 'device_code', 'country_code'])
# print temp_df

# _compare_df(temp_df, cum_temp_df, on=['app_id', 'device_code', 'country_code', 'free_app_download', 'paid_app_download', 'revenue'])
# # diff_df = cum_temp_df.merge(temp_df, indicator=True, how='left', on=['app_id'])
# # print diff_df.T

In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/zidong/aa.store.cum/store_cum_sample_result/")
df.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store.app-est-cum.v1/store_cum_sample_result/", mode="overwrite")

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store.app-est-cum.v1/


In [0]:

import pandas as pd

def get_date_list(start_date, end_date, freq="D"):
    import pandas as pd
    """
    freq:   D: calendar day frequency
            M: month end frequency
            MS: month start frequency
            A, Y: year end frequency
            AS, YS: year start frequency
    """
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


# cum_dates = get_date_list('2010-07-04', '2010-07-31', freq='D') + \
#             get_date_list('2010-08-31', '2020-03-31', freq='M') + \
#             get_date_list('2020-04-01', '2020-04-18', freq='D')  # prod
# cum_dates = get_date_list('2010-07-04', '2010-07-31', freq='D') + \
# get_date_list('2010-08-31', '2011-09-30', freq='M')       # test monthly
# cum_dates = get_date_list('2010-07-04', '2010-07-10', freq='D')
# store_dates = get_date_list('2010-07-04', '2020-04-18', freq='D')  # prod
# store_dates = get_date_list('2010-07-04', '2011-09-30', freq='D')     # test monthly
store_dates = get_date_list('2010-07-04', '2010-08-31', freq='D')

base_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/"

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([StructField("app_id", StringType(), True)])
prev_df = spark.createDataFrame([], schema=schema)


paths = []
print store_dates[0]
for i in range(len(store_dates)):
    path = base_path + "granularity=daily/date=" + store_dates[i] + "/"
    paths.append(path)
# print paths

df = spark.read.option('basePath', base_path).parquet(*paths).select('app_id').distinct()

# compare
cum_df = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/").where("date='{}'".format(store_dates[-1])).select('app_id').distinct()
except_df = df.subtract(cum_df)
if except_df.first():
    print "failed date:", cum_date
print "SUCCESSED"

# save
df.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store.app-est-cum.v1/store_cum_completeness_result/", mode="overwrite")

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store.app-est-cum.v1/store_cum_completeness_result/").show(10)

In [0]:

import pandas as pd

def get_date_list(start_date, end_date, freq="D"):
    import pandas as pd
    """
    freq:   D: calendar day frequency
            M: month end frequency
            MS: month start frequency
            A, Y: year end frequency
            AS, YS: year start frequency
    """
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


# cum_dates = get_date_list('2010-07-04', '2010-07-31', freq='D') + \
#             get_date_list('2010-08-31', '2020-03-31', freq='M') + \
#             get_date_list('2020-04-01', '2020-04-18', freq='D')  # prod
# cum_dates = get_date_list('2010-07-04', '2010-07-31', freq='D') + \
# get_date_list('2010-08-31', '2011-09-30', freq='M')       # test monthly
# cum_dates = get_date_list('2010-07-04', '2010-07-10', freq='D')
# store_dates = get_date_list('2010-07-04', '2020-04-18', freq='D')  # prod
# store_dates = get_date_list('2010-07-04', '2011-09-30', freq='D')     # test monthly
store_dates = get_date_list('2010-07-04', '2010-08-31', freq='D')

base_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/"

df = spark.read.option('basePath', base_path).parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2010-0[7-8]*/").select('app_id').distinct()

# compare
cum_df = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/").where("date='{}'".format(store_dates[-1])).select('app_id').distinct()
except_df = df.subtract(cum_df)
if except_df.first():
    print "failed date:", cum_date
print "SUCCESSED"

# save
df.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store.app-est-cum.v1/store_cum_completeness_result/", mode="overwrite")